## Scrapping LinkedIn Job Postings

In [26]:
#import beautiful soup libs
from bs4 import BeautifulSoup

#import selenium libs
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException   

### 1. Expand Website using Selenium

In [108]:
path = './chromedriver_win32/chromedriver.exe'  
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Singapore&geoId=102454443&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'

# incognito mode
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path=path, options=chrome_options)  
driver.get(url)

# allow webpage to load
time.sleep(5) 

# set waiting time before next scroll
SCROLL_PAUSE_TIME = 1

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

# infinite scrolling
while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for page to load
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
# A 'show more' button appears at random and breaks the loop. Automate button clicks and infinite scrolling
while True:    
        driver.find_element_by_class_name('infinite-scroller__show-more-button').click()
        time.sleep(1)
        
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


### 2. Retrieve web content using beautifulSoup

In [2]:
# Extract data from html 
soup = BeautifulSoup(driver.page_source, 'html.parser')
# soup

### 3. Extract required data fields

In [169]:
jobs = soup.find_all('div', class_='base-card base-card--link base-search-card base-search-card--link job-search-card')

job_title_lst = []
job_url_lst = []
company_lst = []
company_url_lst = []

for each in jobs:
    
    # find job_title
    job_title = each.find('h3', class_='base-search-card__title')
    job_title = job_title.get_text().strip()
    job_title_lst.append(job_title)
    
    #find job url
    job_url = each.find('a', class_='base-card__full-link')['href']
    job_url_lst.append(job_url)
    
    #find company name
    company = each.find('h4', class_='base-search-card__subtitle')
    company = company.find('a').contents[0].strip()
    company_lst.append(company)

    # find company url
    company_url = each.find('h4', class_='base-search-card__subtitle')
    company_url = company_url.find('a')['href'].strip()
    company_url_lst.append(company_url)

    

In [172]:
#store data in dataframe
import numpy as np
import pandas as pd

data={'company_name': company_lst,'job_title':job_title_lst, 'job_url': job_url_lst,  'company_url': company_url_lst}
df = pd.DataFrame(data)

In [174]:
#export data to csv
df.to_csv('output.csv')